# CNF Solved with Quantum Search

The following codes aim to find a solution that makes the CNF formula true.

To start with we import som basic utilities:

In [4]:
from QuICT.core import Circuit
from QuICT.core.gate import *
from QuICT.core.gate.backend import MCTOneAux
from QuICT.simulation.state_vector import ConstantStateVectorSimulator

## Quantum Search: Grover's Algorithm

`QuICT` has built-in implementation of Grover's algorithm. It take an oracle that flips the phase of target states. More explanation of Grover's algorithm can be found on QCQI and BHMT.

In [5]:
from QuICT.algorithm.quantum_algorithm.grover import Grover

## CNF: Oracle Construction

[Conjunctive Normal Form](https://en.wikipedia.org/wiki/Conjunctive_normal_form)(CNF) is a canonical expression in Boolean logic. Next we constructs the oracle circuit for any CNF formula given its description file, and combine the oracle with `Grover` module to find solutions.

This oracle is implemented in `QuICT`.

In [6]:
from QuICT.algorithm.quantum_algorithm.CNF import CNFSATOracle

2022-12-14 14:15:44 | circuit | INFO | Initial Quantum Circuit circuit_bddebdf67b7611edb1e91fdda90babe6 with 1 qubits.
2022-12-14 14:15:44 | circuit | INFO | Initial Quantum Circuit circuit_bddf52de7b7611edb1e91fdda90babe6 with 1 qubits.
2022-12-14 14:15:44 | circuit | INFO | Initial Quantum Circuit circuit_bddf8d587b7611edb1e91fdda90babe6 with 1 qubits.
2022-12-14 14:15:44 | circuit | INFO | Initial Quantum Circuit circuit_bddfd31c7b7611edb1e91fdda90babe6 with 1 qubits.
2022-12-14 14:15:44 | circuit | INFO | Initial Quantum Circuit circuit_bdf640207b7611edb1e91fdda90babe6 with 1 qubits.
2022-12-14 14:15:44 | circuit | INFO | Initial Quantum Circuit circuit_bdf697467b7611edb1e91fdda90babe6 with 1 qubits.
2022-12-14 14:15:44 | circuit | INFO | Initial Quantum Circuit circuit_bdf6c11c7b7611edb1e91fdda90babe6 with 2 qubits.
2022-12-14 14:15:44 | circuit | INFO | Initial Quantum Circuit circuit_bdf6ea667b7611edb1e91fdda90babe6 with 2 qubits.
2022-12-14 14:15:44 | circuit | INFO | Initial Q

The following function reads information from CNF description file:

In [7]:
def read_CNF(cnf_file):
        variable_number = 0
        clause_number = 0
        CNF_data = []
        f = open(cnf_file, 'r') 
        for line in f.readlines():
            new = line.strip().split()
            int_new=[]
            if new[0] == 'p':
                variable_number = int(new[2])
                clause_number = int(new[3])
            else:
                for x in new:
                    if (x != '0') and (int(x) not in int_new):
                        int_new.append(int(x))
                        if (- int(x)) in int_new:
                            int_new = []
                            break
            CNF_data.append(int_new)
        f.close()
        return variable_number, clause_number, CNF_data

Checking CNF solution is neccessary since the algorithm is not deterministic: 

In [8]:
def check_solution(variable_data, variable_number, clause_number, CNF_data):
    cnf_result = 1
    for i in range(clause_number):
        clause_result = 0
        if CNF_data[i+1] == []:
            clause_result = 1
        else:
            for j in range(len(CNF_data[i+1])):
                if CNF_data[i+1][j] > 0:
                    clause_result = clause_result + variable_data[CNF_data[i+1][j]-1]
                else:
                    if CNF_data[i+1][j] < 0:
                        clause_result = clause_result  + ( 1 - variable_data[-CNF_data[i+1][j]-1] )
            if clause_result == 0:
                cnf_result = 0
                break
    if cnf_result == 1:
        return True
    else:
        return False

The combination is as follows, and we try to find solution to a 16-variable CNF formula in `test.cnf`:

In [9]:
file_path = 'test.cnf'
variable_number, clause_number, CNF_data = read_CNF(file_path)

ancilla_qubits_num=5
dirty_ancilla=1
cnf = CNFSATOracle()
cnf.run(file_path, ancilla_qubits_num, dirty_ancilla)

oracle = cnf.circuit()
grover = Grover(ConstantStateVectorSimulator())
    
circ = grover.circuit(variable_number, ancilla_qubits_num + dirty_ancilla, oracle, n_solution=1, measure=False, is_bit_flip=True)
print(f"constrcution finished.")
grover.simulator.run(circ)
print(f"simulation   finished.")
n_hit = 0
n_all = 1000
result_samples = grover.simulator.sample(n_all)
print(f"sampling     finished.")

2022-12-14 14:15:45 | circuit | INFO | Initial Quantum Circuit circuit_be9e265a7b7611edb1e91fdda90babe6 with 14 qubits.
2022-12-14 14:15:54 | Grover | INFO | circuit width          =   14
oracle  calls          =   12
other circuit size     =  920

constrcution finished.
simulation   finished.
sampling     finished.


We can check the success rate:

In [10]:
result_var_samples = np.array(result_samples).reshape((1<<variable_number,1<<(ancilla_qubits_num + dirty_ancilla))).sum(axis=1)
for result in range(1<<variable_number):
    result_str = bin(result)[2:].rjust(variable_number,'0')
    if check_solution([int(x) for x in result_str], variable_number, clause_number, CNF_data):
        n_hit += result_var_samples[result]
print(f"[{n_hit}/{n_all}]:{n_hit/n_all:.3f}")

[1000/1000]:1.000
